In [1]:
import contextlib
import glob
import hashlib
import mimetypes
import os
import re
import tempfile
import zipfile
from datetime import datetime, timezone
from pathlib import Path

import aiofiles
import bittensor
import uvicorn
from fastapi import FastAPI, File, Form, Header, HTTPException, Request, UploadFile
from fastapi.responses import FileResponse, JSONResponse
from pydantic import BaseModel

from mycelia.shared.app_logging import configure_logging, structlog
from mycelia.shared.chain import _subtensor_lock
from mycelia.shared.checkpoint import (
    delete_old_checkpoints_by_hotkey,
    get_resume_info,
)
from mycelia.shared.config import ValidatorConfig, parse_args
from mycelia.shared.schema import (
    construct_block_message,
    construct_model_message,
    verify_message,
)

from mycelia.shared.client import download_model, submit_model
from mycelia.shared.chain import *
from mycelia.shared.app_logging import configure_logging
from mycelia.shared.checkpoint import (
    get_resume_info,
)
from mycelia.shared.config import MinerConfig
import bittensor 

configure_logging()

miner_config = MinerConfig.from_path("/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/config.yaml")
validator_config = MinerConfig.from_path("/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/config.yaml")
wallet, subtensor = setup_chain_worker(miner_config)

In [4]:
url = "149.137.225.62:8001"

out_dir=config.ckpt.validator_checkpoint_path / f"uid{5}"


out_dir.mkdir(parents=True, exist_ok=True)
download_model(
    url=f"http://{url}/get-checkpoint",
    my_hotkey=wallet.hotkey,  # type: ignore
    target_hotkey_ss58="5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH",
    block=subtensor.block,
    expert_group_ids=[config.task.expert_group_id],
    token=getattr(config.cycle, "token", ""),
    out_dir=out_dir,
)

2025-12-16 01:25:23 [info     ] out is not zip               [mycelia.shared.client] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/miner/validator_checkpoint/uid5'),)
Done:       22.00 B / 22.00 B (100.0%) in 0.0s @ 14.19 KiB/s [mycelia.shared.client]


In [2]:

config = validator_config
expert_group_ids = [0, 1]

logger.info(f'zipping {expert_group_ids}')
resume, model_meta, latest_checkpoint_path = get_resume_info(rank=0, config=config)
logger.info("checkpoint, C", latest_checkpoint_path)

if not latest_checkpoint_path:
    raise HTTPException(status_code=500, detail="CHECKPOINT_PATH env var is not set")

ckpt_dir = latest_checkpoint_path  # directory that contains group_*.pt / shared*.pt
expert_group_ids.sort()

# Deterministic zip name for this combination of groups + step
zip_name = f"expert_group_step{model_meta.global_ver}_{','.join(str(x) for x in expert_group_ids)}.zip"
zip_path = os.path.join(ckpt_dir, zip_name)

logger.info('zip path', zip_path=zip_path, zip_name=zip_name)
# # If the zip already exists, skip re-creating it
# if os.path.exists(zip_path):
#     return FileResponse(
#         zip_path,
#         filename=zip_name,
#         media_type="application/zip",
#     )

model_files = []
for expert_group_id in expert_group_ids:
    group_pattern = os.path.join(ckpt_dir, f"model_expgroup_{expert_group_id}.pt")
    model_files += glob.glob(group_pattern)


shared_pattern = os.path.join(ckpt_dir, "model_shared.pt")
model_files += glob.glob(shared_pattern)

# Create a temp dir for the zip
tmp_dir = tempfile.mkdtemp(
    prefix=zip_name.replace(".zip", "_"),
    dir=ckpt_dir,
)

zip_name = f"expert_group_{','.join(str(x) for x in expert_group_ids)}.zip"
zip_path = os.path.join(tmp_dir, zip_name)

# Write all related files into the zip (group + shared)
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
    for path in model_files:
        arcname = os.path.basename(path)
        zf.write(path, arcname=arcname)

2025-12-16 01:43:25 [info     ] zipping [0, 1]               [mycelia.shared.chain]
2025-12-16 01:43:25 [info     ] Looking for checkpoint from folder [mycelia.shared.checkpoint] path={PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation')} result=found
2025-12-16 01:43:25 [info     ] checkpoint, C                [mycelia.shared.chain] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/globalver_3'),)
2025-12-16 01:43:25 [info     ] zip path                     [mycelia.shared.chain] zip_name=expert_group_step3_0,1.zip zip_path=/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/globalver_3/expert_group_step3_0,1.zip
